In [1]:
%load_ext autoreload
%autoreload 2


from util_0630 import *
from myalgorithm_0630_1 import algorithm

- 실험해볼 내용들

1. 가까운 거리는 먼저 WALK를 배정하면 어떨까? -> try_merging_bundles_by_dist_walk_prefered 함수 제작 후 실험 결과 결과가 꼭 개선된다는 보장이 없었음
2. 배달원을 전부 한번에 재할당해보면 어떨까? -> check_reassign_riders 함수 제작 후 실험 결과 기존의 배달원 재할당 코드보다 확실히 효과적이었음
3. 가까운 번들 내의 원소를 교환하는 휴리스틱 코드를 만들면 어떨까?
4. 만약 번들의 각 주문을 CAR로 나누어 할당하였을 때 더 비용이 최소화가 된다면 나누는 언번들링 코드를 제작해보면 어떨까?

- 코드 구조상 개선할 부분

1. get_init_bundle 함수를 util로 옮기기 -> 완료
2. 배달원 커스텀 할당 기능을 util로 옮기기 -> 완료

In [2]:

problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = '../alg_test_problems_20240429/TEST_K100_2.json'

# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

## --------------- 초기 번들링 최적화 코드 --------------------------

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

min_init_cost = inf
min_init_cost_bundle = []
min_init_cost_rider_availables = []

# try_merging_bundles_by_dist // 모든 배달원과 모든 조합에서의 최소 거리를 보는 케이스
# try_merging_bundles_by_dist_walk_prefered // 위와 동일하지만 우선적으로 도보 배달원을 할당하는 케이스
# try_merging_bundles_by_cost // 번들 합칠 때 거리가 아닌 비용을 보는 케이스

for weight1 in [0, 1]:
    for weight2 in [-3, -2, -1.5, -1, -0.5, 0, 0.5]:
        bundles, result_rider_availables, cost = get_init_bundle(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, try_merging_bundles_by_dist, 4)

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundles = bundles
            min_init_cost_rider_availables = result_rider_availables

for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = min_init_cost_rider_availables[rider_i]
all_bundles = min_init_cost_bundles


## ----------------- 기본 배달원 재할당 코드 -------------------

for bundle in all_bundles:
    new_rider = get_cheaper_available_riders(ALL_RIDERS, bundle.rider)
    if new_rider is not None:
        old_rider = bundle.rider

        check_result = check_bundle_rider_changing(ALL_ORDERS, bundle, new_rider)
        if check_result:
            bundle.shop_seq = check_result[0]
            bundle.dlv_seq = check_result[1]
            bundle.rider = check_result[2]
            bundle.update_cost()

            old_rider.available_number += 1
            new_rider.available_number -= 1


## ----------------- 커스텀 배달원 재할당 코드 -----------------------

all_bundles, rider_availables = check_reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])


3691.1


In [ ]:
3761

### 번들을 CAR 또는 WALK 배달원이 할당된 각각의 주문으로 분리하였을 때 비용이 줄어드는지 확인하고 분리하기

In [57]:
new_all_bundles = []

for bundle in all_bundles:
    if len(bundle.shop_seq) == 1:
        continue

    former_cost = bundle.cost

    new_total_cost = 0
    for order in bundle.shop_seq:
        one_order_bundle = Bundle(ALL_RIDERS, car_rider, [order], [order], ALL_ORDERS[order].volume, DIST[order][order + K])

        new_total_cost += one_order_bundle.cost

    print(former_cost, new_total_cost, bundle.rider.type, len(bundle.shop_seq))

    if new_total_cost < former_cost:
        pass
        # print(former_cost - new_total_cost, bundle.rider.type, len(bundle.shop_seq))

7635.200000000001 19533.0 BIKE 3
8070.200000000001 20322.0 BIKE 3
11045.0 21814.0 BIKE 3
8552.0 18799.0 BIKE 3
8579.0 19997.0 BIKE 3
8889.2 22375.0 BIKE 3
9517.0 21559.0 CAR 3
9710.6 19105.0 BIKE 3
9238.0 18648.0 CAR 3
8618.0 24871.0 BIKE 3
7331.6 20363.0 BIKE 3
9553.0 13428.0 CAR 2
11342.0 19237.0 CAR 3
12006.0 23800.0 CAR 3
9153.2 23235.0 BIKE 3
7577.0 23024.0 BIKE 3
10495.0 18038.0 CAR 3
11072.599999999999 22200.0 BIKE 3
8197.4 23937.0 BIKE 3
7938.799999999999 16535.0 BIKE 2
10159.4 16756.0 BIKE 2
9322.400000000001 19242.0 BIKE 3
8296.4 18243.0 BIKE 3
7376.6 19768.0 BIKE 3
8370.2 23960.0 BIKE 3
8525.6 24017.0 BIKE 3
6771.0 11565.0 CAR 2
9899.0 14144.0 CAR 2
9654.0 15195.0 CAR 2
9781.0 12389.0 CAR 2
9861.0 21471.0 CAR 3
8887.0 13209.0 CAR 2
7623.0 13952.0 CAR 2
8178.0 12013.0 CAR 2


In [68]:
# WALK 배달원의 수 제한과 상관 없이 전부 WALK 배달원으로 할당해봄
new_all_bundles = []

for bundle in all_bundles:
    if len(bundle.shop_seq) == 1:
        continue

    former_cost = bundle.cost

    new_total_cost = 0
    for order in bundle.shop_seq:
        one_order_bundle = Bundle(ALL_RIDERS, walk_rider, [order], [order], ALL_ORDERS[order].volume, DIST[order][order + K])

        new_total_cost += one_order_bundle.cost

    print(former_cost, new_total_cost, bundle.rider.type, len(bundle.shop_seq))

    if new_total_cost < former_cost:
        pass
        # print(former_cost - new_total_cost, bundle.rider.type, len(bundle.shop_seq))

7635.200000000001 16359.9 BIKE 3
8070.200000000001 16596.600000000002 BIKE 3
11045.0 17044.2 BIKE 3
8552.0 16139.7 BIKE 3
8579.0 16499.1 BIKE 3
8889.2 17212.5 BIKE 3
9517.0 16967.699999999997 CAR 3
9710.6 16231.5 BIKE 3
9238.0 16094.4 CAR 3
8618.0 17961.3 BIKE 3
7331.6 16608.9 BIKE 3
9553.0 11028.400000000001 CAR 2
11342.0 16271.1 CAR 3
12006.0 17640.0 CAR 3
9153.2 17470.5 BIKE 3
7577.0 17407.2 BIKE 3
10495.0 15911.4 CAR 3
11072.599999999999 17160.0 BIKE 3
8197.4 17681.1 BIKE 3
7938.799999999999 11960.5 BIKE 2
10159.4 12026.8 BIKE 2
9322.400000000001 16272.6 BIKE 3
8296.4 15972.900000000001 BIKE 3
7376.6 16430.4 BIKE 3
8370.2 17688.0 BIKE 3
8525.6 17705.1 BIKE 3
6771.0 10469.5 CAR 2
9899.0 11243.2 CAR 2
9654.0 11558.5 CAR 2
9781.0 10716.7 CAR 2
9861.0 16941.300000000003 CAR 3
8887.0 10962.7 CAR 2
7623.0 11185.599999999999 CAR 2
8178.0 10603.9 CAR 2


> 결과적으로 번들을 묶는 것이 이득이 아닌 경우가 없음을 발견하였다.

### 4번들 가능 여부 확인

- 먼저 주문 2개씩만 가진 번들까지만 만든 다음에 주문 4개씩 가진 번들을 시도해봄

In [69]:
get_init_bundle_4_order_bundle_maker

<function util_0630.get_init_bundle_4_order_bundle_maker(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2)>

In [5]:

problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'
# problem_file = '../alg_test_problems_20240429/TEST_K100_2.json'

## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

## --------------- 초기 번들링 최적화 코드 --------------------------

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

min_init_cost = inf
min_init_cost_bundle = []
min_init_cost_rider_availables = []

# try_merging_bundles_by_dist // 모든 배달원과 모든 조합에서의 최소 거리를 보는 케이스
# try_merging_bundles_by_dist_walk_prefered // 위와 동일하지만 우선적으로 도보 배달원을 할당하는 케이스
# try_merging_bundles_by_cost // 번들 합칠 때 거리가 아닌 비용을 보는 케이스

weight1 = 1
weight2 = -1.5

all_bundles, result_rider_availables, cost = get_init_bundle_4_order_bundle_maker(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2)

for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = result_rider_availables[rider_i]

## ----------------- 기본 배달원 재할당 코드 -------------------

for bundle in all_bundles:
    new_rider = get_cheaper_available_riders(ALL_RIDERS, bundle.rider)
    if new_rider is not None:
        old_rider = bundle.rider

        check_result = check_bundle_rider_changing(ALL_ORDERS, bundle, new_rider)
        if check_result:
            bundle.shop_seq = check_result[0]
            bundle.dlv_seq = check_result[1]
            bundle.rider = check_result[2]
            bundle.update_cost()

            old_rider.available_number += 1
            new_rider.available_number -= 1


## ----------------- 커스텀 배달원 재할당 코드 -----------------------

all_bundles, rider_availables = check_reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])

3542.1769999999997


### Weight search도 다시 반영해줌

In [6]:

problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = '../alg_test_problems_20240429/TEST_K100_2.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'
# problem_file = '../alg_test_problems_20240429/TEST_K100_2.json'

## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

## --------------- 초기 번들링 최적화 코드 --------------------------

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

min_init_cost = inf
min_init_cost_bundle = []
min_init_cost_rider_availables = []

# try_merging_bundles_by_dist // 모든 배달원과 모든 조합에서의 최소 거리를 보는 케이스
# try_merging_bundles_by_dist_walk_prefered // 위와 동일하지만 우선적으로 도보 배달원을 할당하는 케이스
# try_merging_bundles_by_cost // 번들 합칠 때 거리가 아닌 비용을 보는 케이스

for weight1 in [0, 1]:
    for weight2 in [-3, -2, -1.5, -1, -0.5, 0, 0.5]:
        bundles, result_rider_availables, cost = get_init_bundle_4_order_bundle_maker(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2)

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundles = bundles
            min_init_cost_rider_availables = result_rider_availables

for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = min_init_cost_rider_availables[rider_i]
all_bundles = min_init_cost_bundles


## ----------------- 기본 배달원 재할당 코드 -------------------

for bundle in all_bundles:
    new_rider = get_cheaper_available_riders(ALL_RIDERS, bundle.rider)
    if new_rider is not None:
        old_rider = bundle.rider

        check_result = check_bundle_rider_changing(ALL_ORDERS, bundle, new_rider)
        if check_result:
            bundle.shop_seq = check_result[0]
            bundle.dlv_seq = check_result[1]
            bundle.rider = check_result[2]
            bundle.update_cost()

            old_rider.available_number += 1
            new_rider.available_number -= 1


## ----------------- 커스텀 배달원 재할당 코드 -----------------------

all_bundles, rider_availables = check_reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])

3487.3210000000004


> 추가적인 실험 결과 번들로 묶는 최대 주문 수를 2 -> 4 -> 3으로 했을 때 바로 4개로 했을 때보다 오히려 시간이 적게 걸리면서도 비용이 낮았음.

## 제출용

In [6]:
problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'

timelimit = 60

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])

solution = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

In [7]:
with open(problem_file, 'r') as f:
    prob = json.load(f)
K = prob['K']
ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]
DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

checked_solution['avg_cost']

3487.3210000000004